In [1]:
import pandas as pd


In [12]:
df = pd.read_csv("./data/train.csv")

In [13]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [14]:
df.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


In [15]:
df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [17]:
df.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [72]:
# Obtain distance between two points on earth
# https://en.wikipedia.org/wiki/Great-circle_distance

# Below function gives distance in KM between to points on the earth surface identified by
# their longitude and latitudes (from_long, from_lat) & (to_long, to_lat)
# 0.009 of difference in x co-ordinate = 1.002 km
# 0.009 of difference in y co-ordinate = 1.002 km

from math import sin, cos, acos

def get_distance(from_long, from_lat, to_long, to_lat):
    
    if from_long == to_long and from_lat == to_lat: 
        return 0
    
    v_pi             = 3.1415926; 
    v_earth_radius   = 6378; # Radius of the Earth in km    
    v_from_x_radians = (v_pi / 180) * from_long
    v_from_y_radians = (v_pi / 180) * from_lat
    v_to_x_radians   = (v_pi / 180) * to_long
    v_to_y_radians   = (v_pi / 180) * to_lat

    v_distance = ((acos(sin(v_from_y_radians) * sin(v_to_y_radians) 
                       + (cos(v_from_y_radians) * cos(v_to_y_radians) *  cos(v_from_x_radians - v_to_x_radians)))) 
                  * v_earth_radius)
    return v_distance # Unit in kilometer


In [73]:
get_distance(103.903806, 1.404605, 103.908988, 1.400917)

0.7078785036170167

In [74]:
df.apply(lambda r: get_distance(r['pickup_longitude'], r['pickup_latitude'], r['dropoff_longitude'], r['dropoff_latitude']), axis=1)

0           1.500167
1           1.807491
2           6.392114
3           1.487131
4           1.189894
5           1.100150
6           1.327736
7           5.721260
8           1.311793
9           5.126788
10          3.810321
11          3.777242
12          1.861526
13          0.992774
14          6.389849
15          0.657299
16          3.431852
17          2.541461
18          4.610261
19          1.304703
20          2.508679
21          1.726445
22          2.069356
23          4.880148
24         20.625211
25          4.564535
26          6.062763
27          3.742850
28          2.527623
29          9.950364
             ...    
1458614     1.344135
1458615     2.769610
1458616     0.981149
1458617     0.656388
1458618     1.173294
1458619     2.262217
1458620     4.144866
1458621     4.408940
1458622     0.730553
1458623     1.670861
1458624     0.780202
1458625     3.663272
1458626     1.287039
1458627     2.180538
1458628     1.360271
1458629     2.562262
1458630     1

In [68]:
df.loc[1620]

id                              id0753277
vendor_id                               1
pickup_datetime       2016-01-27 22:29:31
dropoff_datetime      2016-01-27 22:29:58
passenger_count                         1
pickup_longitude                 -74.2099
pickup_latitude                   40.8169
dropoff_longitude                -74.2099
dropoff_latitude                  40.8169
store_and_fwd_flag                      N
trip_duration                          27
Name: 1620, dtype: object

In [70]:
get_distance(-74.2099, 40.8169, -74.2099, 40.8169)

9.503960609436035e-05

In [71]:
if 1.0 == 1.0:
    print ('AS')

AS
